# Optimizing Model Parameters

In [1]:
# Prerequisite Code

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data", train=True, download=True, transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data", train=False, download=True, transform=ToTensor()
)

train_dl = DataLoader(training_data, batch_size=64)
test_dl = DataLoader(test_data, batch_size=1)

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.flattent = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )
    
    def forward(self, x):
        x = self.flattent(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NN()

In [2]:
# Hyperparameters
lr = 1e-3
bs = 64
epochs = 10

In [3]:
# Optimization Loop

# Loss Fucntion
loss_fn = nn.CrossEntropyLoss()

# Opimizer
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [4]:
# Full Implementation

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg Loss: {test_loss:>8f}\n")

In [5]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------------")
    train_loop(train_dl, model, loss_fn, optimizer)
    test_loop(test_dl, model, loss_fn)
print("Done")

Epoch 1
-------------------------------------
loss: 2.301236 [    0/  938]
loss: 2.293142 [ 6400/  938]
loss: 2.271483 [12800/  938]
loss: 2.268280 [19200/  938]
loss: 2.244386 [25600/  938]
loss: 2.214475 [32000/  938]
loss: 2.220053 [38400/  938]
loss: 2.185868 [44800/  938]
loss: 2.179757 [51200/  938]
loss: 2.150800 [57600/  938]
Test Error: 
 Accuracy: 52.1%, Avg Loss: 2.147123

Epoch 2
-------------------------------------
loss: 2.155396 [    0/  938]
loss: 2.149716 [ 6400/  938]
loss: 2.088409 [12800/  938]
loss: 2.107355 [19200/  938]
loss: 2.048808 [25600/  938]
loss: 1.991027 [32000/  938]
loss: 2.014002 [38400/  938]
loss: 1.934425 [44800/  938]
loss: 1.933880 [51200/  938]
loss: 1.870468 [57600/  938]
Test Error: 
 Accuracy: 59.7%, Avg Loss: 1.866471

Epoch 3
-------------------------------------
loss: 1.896159 [    0/  938]
loss: 1.869167 [ 6400/  938]
loss: 1.752993 [12800/  938]
loss: 1.796953 [19200/  938]
loss: 1.673800 [25600/  938]
loss: 1.639437 [32000/  938]
loss: 